In [ ]:
# read from a zipped CSV and avoid UnicodeDecodeError by using a permissive encoding
#df = pd.read_csv('GlobalTerrorismData.zip', compression='zip', encoding='latin-1')

In [9]:
#Use matplotlib to import libraries & define functions for plotting data.
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np #linear algebra
import os #accessing directory structure
import pandas as pd 
# read from a zipped CSV and avoid UnicodeDecodeError by using a permissive encoding


In [10]:
import zipfile

zip_path = 'GlobalTerrorismData.zip'

if os.path.exists(zip_path):
	if zipfile.is_zipfile(zip_path):
		with zipfile.ZipFile(zip_path) as z:
			print(z.namelist())
	else:
		try:
			# fall back in case it's actually a directory
			print(os.listdir(zip_path))
		except NotADirectoryError:
			print(f"'{zip_path}' exists but is not a zip file nor a directory.")
else:
	print(f"'{zip_path}' not found.")

['globalterrorismdb_0718dist.csv']


In [11]:
#Distribution graphs (histograms/bar graphs) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6*nGraphPerRow, 8*nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDF = df.iloc[:, i]
        if (not np.issubdtype(type(columnDF.iloc[0]), np.number)):
            valueCounts = columnDF.value_counts()
            valueCounts.plot.bar()
        else:
            columnDF.hist()
        plt.ylabel('counts')
        plt.xticks(rotation=90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()